In [1]:
import numpy as np
from src.NN import NN
import src.utils as utils

# Settings
csv_filename = "data/seeds_dataset.csv"
hidden_layers = [1,2] # number of nodes in hidden layers i.e. [layer1, layer2, ...]
eta = 0.1 # learning rate
n_epochs = 50 # number of training epochs
n_folds = 4 # number of folds for cross-validation
seed_crossval = 1 # seed for cross-validation
seed_weights = 1 # 

In [2]:

print("Reading '{}'...".format(csv_filename))
X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)

Reading 'data/seeds_dataset.csv'...


In [3]:
print(np.hsplit(X,7)[0])

[[ 8.76678659e-02]
 [-4.43482563e-02]
 [-2.49320656e-01]
 [-4.05655538e-01]
 [ 3.93389412e-01]
 [-2.18053680e-01]
 [-1.10356317e-01]
 [-3.53543911e-01]
 [ 2.37054530e-01]
 [-1.06882209e-01]
 [-7.94755687e-01]
 [-2.94484067e-01]
 [-3.11854609e-01]
 [ 3.03062592e-01]
 [-1.01709863e+00]
 [ 8.15153945e-04]
 [ 4.10759954e-01]
 [-6.90532433e-01]
 [-7.87807471e-01]
 [-3.11854609e-01]
 [-5.41145769e-01]
 [-6.41894914e-01]
 [ 1.67572361e-01]
 [-3.18802826e-01]
 [-3.70914453e-01]
 [ 1.47115879e-02]
 [ 3.86441195e-01]
 [ 4.14234063e-01]
 [ 7.19955609e-01]
 [-7.21411241e-02]
 [-2.52794765e-01]
 [-5.09878792e-01]
 [-5.23775226e-01]
 [-6.41894914e-01]
 [ 1.71046469e-01]
 [ 3.55562387e-02]
 [-4.19551972e-01]
 [ 1.22408951e-01]
 [-6.13306301e-03]
 [-7.56152326e-02]
 [-5.12964732e-02]
 [-2.00683138e-01]
 [ 2.68321507e-01]
 [-1.79838487e-01]
 [-2.35424223e-01]
 [-1.69416161e-01]
 [ 7.76337090e-03]
 [-1.90260812e-01]
 [-3.04518224e-02]
 [ 1.29357168e-01]
 [-1.00667630e+00]
 [-1.24638979e+00]
 [-1.3123978

In [4]:
m,n=X.shape
n

7

In [5]:
def get_splits(X):
    m,n=X.shape
    splitWeights=list()
    splits = np.hsplit(X,n)
    # return splits

    for o,oX in enumerate(splits):
        N, d = oX.shape

        print("Neural network model:")
        print(" input_dim = {}".format(d))
        print(" hidden_layers = {}".format(hidden_layers))
        print(" output_dim = {}".format(n_classes))
        print(" eta = {}".format(eta))
        print(" n_epochs = {}".format(n_epochs))
        print(" n_folds = {}".format(n_folds))
        print(" seed_crossval = {}".format(seed_crossval))
        print(" seed_weights = {}\n".format(seed_weights))

        # Create cross-validation folds
        idx_all = np.arange(0, N)
        idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

        # Train/evaluate the model on each fold
        acc_train, acc_valid = list(), list()
        print("Cross-validating with {} folds...".format(len(idx_folds)))
        for i, idx_valid in enumerate(idx_folds):

            # Collect training and test data from folds
            idx_train = np.delete(idx_all, idx_valid)
            X_train, y_train = oX[idx_train], y[idx_train]
            X_valid, y_valid = oX[idx_valid], y[idx_valid]

            # Build neural network classifier model and train
            model = NN(input_dim=d, output_dim=n_classes,
                        hidden_layers=hidden_layers, seed=seed_weights)
            model.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

            # Make predictions for training and test data
            ypred_train = model.predict(X_train)
            ypred_valid = model.predict(X_valid)

            # Compute training/test accuracy score from predicted values
            acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
            acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

            # Print cross-validation result
            print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

        # Print results
        print(o," : ")
        print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

        splitWeights.append(model.get_weights())
    return splitWeights

In [6]:
wt = get_splits(X)

Neural network model:
 input_dim = 1
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...
 Fold 1/4: acc_train = 34.07%, acc_valid = 18.18% (n_train = 135, n_valid 44)
 Fold 2/4: acc_train = 38.52%, acc_valid = 27.27% (n_train = 135, n_valid 44)
 Fold 3/4: acc_train = 62.96%, acc_valid = 75.00% (n_train = 135, n_valid 44)
 Fold 4/4: acc_train = 39.26%, acc_valid = 25.00% (n_train = 135, n_valid 44)
0  : 
  -> acc_train_avg = 43.70%, acc_valid_avg = 36.36%
Neural network model:
 input_dim = 1
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...
 Fold 1/4: acc_train = 68.15%, acc_valid = 59.09% (n_train = 135, n_valid 44)
 Fold 2/4: acc_train = 65.93%, acc_valid = 65.91% (n_train = 135, n_valid 44)
 Fold 3/4: acc_train = 62.96%, acc_valid = 75.00% (n_train = 135, n_valid 44)
 Fold 4/4: acc_train =

In [7]:
(wt [1][1])

array([[ 2.259065 ],
       [-1.2133725]], dtype=float32)

In [8]:
# len(model.get_weights()[0][])
wt[2]

array([array([[1.5162259]], dtype=float32),
       array([[0.32234785],
       [1.4898083 ]], dtype=float32),
       array([[-1.137341  , -0.4418237 ],
       [-0.5489515 , -0.08448908],
       [-0.17905736, -1.0719728 ]], dtype=float32)], dtype=object)

In [69]:
w1 = get_splits(X)

Neural network model:
 input_dim = 1
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...
 Fold 1/4: acc_train = 68.15%, acc_valid = 59.09% (n_train = 135, n_valid 44)
 Fold 2/4: acc_train = 65.93%, acc_valid = 65.91% (n_train = 135, n_valid 44)
 Fold 3/4: acc_train = 31.85%, acc_valid = 25.00% (n_train = 135, n_valid 44)
 Fold 4/4: acc_train = 66.67%, acc_valid = 63.64% (n_train = 135, n_valid 44)
0  : 
  -> acc_train_avg = 58.15%, acc_valid_avg = 53.41%
Neural network model:
 input_dim = 1
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...
 Fold 1/4: acc_train = 68.15%, acc_valid = 59.09% (n_train = 135, n_valid 44)
 Fold 2/4: acc_train = 38.52%, acc_valid = 27.27% (n_train = 135, n_valid 44)
 Fold 3/4: acc_train = 31.85%, acc_valid = 25.00% (n_train = 135, n_valid 44)
 Fold 4/4: acc_train =

In [87]:
wt[1]

array([array([[2.5309176]], dtype=float32),
       array([[ 2.259065 ],
       [-1.2133725]], dtype=float32),
       array([[-1.173157  , -0.22324422],
       [ 0.8682776 , -2.1692107 ],
       [-1.5540261 ,  0.7542186 ]], dtype=float32)], dtype=object)

In [90]:
new=[]
print("wth",new)
wx=w1
for i,mx in enumerate(wt):
    # m2 = wt[i]
    if i==0:
        print("hitting new")
        new = mx
    else:
        for j,lx in enumerate(mx):
            if j==0 and i!=0:
                print(new[j],"88",lx)
                new[j] = np.append(new[j],lx)
            else:
                new[j] = new[j] + lx

for k in range(len(new)):
    if not k == 0:
        print(k)
        new[k] = np.divide(new[k],len(wt))
        # print(type(l1))
print( new )

wth []
hitting new
[2.0011792 2.5309176 1.5162259 1.863346  1.1245931 1.3149298 2.6542094] 88 [[2.5309176]]
[2.0011792 2.5309176 1.5162259 1.863346  1.1245931 1.3149298 2.6542094
 2.5309176] 88 [[1.5162259]]
[2.0011792 2.5309176 1.5162259 1.863346  1.1245931 1.3149298 2.6542094
 2.5309176 1.5162259] 88 [[1.863346]]
[2.0011792 2.5309176 1.5162259 1.863346  1.1245931 1.3149298 2.6542094
 2.5309176 1.5162259 1.863346 ] 88 [[1.1245931]]
[2.0011792 2.5309176 1.5162259 1.863346  1.1245931 1.3149298 2.6542094
 2.5309176 1.5162259 1.863346  1.1245931] 88 [[1.3149298]]
[2.0011792 2.5309176 1.5162259 1.863346  1.1245931 1.3149298 2.6542094
 2.5309176 1.5162259 1.863346  1.1245931 1.3149298] 88 [[2.6542094]]
1
2
[array([2.0011792, 2.5309176, 1.5162259, 1.863346 , 1.1245931, 1.3149298,
       2.6542094, 2.5309176, 1.5162259, 1.863346 , 1.1245931, 1.3149298,
       2.6542094], dtype=float32)
 array([[-0.01162633],
       [ 1.0305294 ]], dtype=float32)
 array([[-0.42398414, -0.9957853 ],
       [-0.

In [91]:
len(new[0])

13

In [68]:
np.append(w1[0][0],wt[0][0])

array([2.1837533e-05, 2.0011792e+00], dtype=float32)